In [1]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from re import sub
from pandas import DataFrame
from pymystem3 import Mystem
import texterra
from pymorphy2 import MorphAnalyzer
from itertools import chain
from pickle import dump, load
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.cluster import SpectralClustering, AgglomerativeClustering, KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import *
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from sklearn.preprocessing import Normalizer

from scipy.spatial.distance import cosine
import scipy.sparse as sp
import matplotlib.pyplot as plt
import numpy as np

API_KEY = '9988cfb979b80264baeba1386cc7e455f99f943c'

stop = stopwords.words('russian')
la = np.linalg
morph = MorphAnalyzer()
m = Mystem()
# t = texterra.API(API_KEY)
alpha_tokenizer = RegexpTokenizer('[A-Za-zА-Яа-я]\w+')

Using Theano backend.


In [2]:
from pandas import DataFrame, concat
from os import path
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches
# import seaborn

# Домашнее задание 1 [10 баллов] 
# До 1.12.17 23:59

Задание выполняется в группе (1-4 человека). В случае использования какого-либо строннего источника информации обязательно дайте на него ссылку (поскольку другие тоже могут на него наткнуться). Плагиат наказывается нулём баллов за задание и предвзятым отношением в будущем.

Не все части обязательны для выполнения, однако вы можете быть дополнительно оштрафованы за небрежное за выполнение одной или двух частей вместо четырех.

При возниконовении проблем с выполнением задания обращайтесь с вопросами к преподавателю. Поэтому настоятельно рекомендуется выполнять задание заранее, оставив запас времени на всевозможные технические проблемы. Если вы начали читать условие в последний вечер и не успели из-за проблем с установкой какой-либо библиотеки — это ваши проблемы.


Результат выполнения задания — это отчёт в формате html на основе Jupyter Notebook. Нормальный отчёт должен включать в себя:
* Краткую постановку задачи и формулировку задания
* Описание **минимума** необходимой теории и/или описание используемых инструментов - не стоит переписывать лекции или Википедию
* Подробный пошаговый рассказ о проделанной работе
* Аккуратно оформленные результаты
* **Внятные выводы** – не стоит относится к домашнему заданию как к последовательности сугубо технических шагов, а стоит относится скорее как к небольшому практическому исследованию, у которого есть своя цель и свое назначение.

Небрежное его оформление отчета существенно отразится на итоговой оценке. Весь код из отчёта должен быть воспроизводимым, если для этого нужны какие-то дополнительные действия, установленные модули и т.п. — всё это должно быть прописано в тексте в явном виде.

Сдача отчетов осуществляется через систему AnyTask.



## Кластеризация новостей

Входная коллекция данных состоит из двух частей:
1. events.csv – список 28 резонансных событий первой половины 2017 года, каждому событию присвоен свой порядковый номер (id)
2. raw_news.csv – тексты новостей из различных новостных источников, известно, к какому резонансному событию относится каждая новость (столбец event_id).

Будем считать, что одно событие – это один кластер. В этом домашнем задании вам предстоит:
1. провести кластеризацию текстов новостей и проверить, получается ли восстановить кластерную структуру 
2. проверить, можно ли использовать кластерный анализ для обобщения: найти небольшое число кластеров и проверить, получается ли выделить общие направлени новостей

In [9]:
df_events = DataFrame.from_csv('events.csv')
df_news = DataFrame.from_csv('raw_news.csv')

In [3]:
df_news.head()

,event_id,text
0,1,В ПЕТЕРБУРГЕ ПРОШЕЛ МИТИНГ ПРОТИВ ПЕРЕДАЧИ ИС...
1,1,"Lenta.co, Москва, 14 января 2017 СИТУАЦИЯ С П..."
2,1,"Аргументы и Факты (aif.ru), Москва, 14 января..."
3,1,"Google Новости ТОП, Москва, 14 января 2017 АК..."
4,1,"Газета.Ru, Москва, 13 января 2017 В МОСКОВСКО..."


### Часть 1 [2 балла] Предварительная обработка текстов
Проведите предобработку новостей: токенизацию, приведение к нижнему регистру, лемматизацию. Проверьте, есть ли в коллекции дубликаты. Посчитайте, сколько новостей относится к каждому резонансному событию. 

Для начала смерджим все тексты новостей для того, чтобы получить один единый корпус, с которым можно работать:

In [6]:
texts = ' '.join(df_news.text.values)

Очистка новостей от шумов (пока реализовано только удаление УРЛОВ):

In [7]:
def remove_url(text):
    return sub(r'http\S+', '', text)

In [8]:
texts = remove_url(texts)

Токенизация (удаляем всё, что не является alphanumeric-символами):

In [9]:
tokens = alpha_tokenizer.tokenize(texts)

Лемматизация (и автоматические приведение к нижнему регистру). Мы решили рассмотреть три варианта лемматизации, поскольку не все способны одинаково хорошо справляться с контекстной омонимией (как, например, во фразе "Запотело стекло. Варенье стекло по краю банки."):

pymorphy2

In [4]:
# norm_tokens = [morph.parse(word)[0].normal_form for word in tokens]

with open('lemmas_from_texterra.pickle', 'rb') as f:
    texts = load(f)

Сколько новостей относится к каждому резонансному событию?

In [11]:
df_news.event_id.value_counts()

17    102
28    100
18    100
7     100
10    100
12    100
27    100
16    100
1     100
25    100
26    100
21    100
23    100
3      84
22     82
9      82
24     62
4      62
2      51
11     49
19     45
6      41
8      27
13     24
20      8
15      7
5       2
14      2
Name: event_id, dtype: int64

Есть ли в коллекции дубликаты?

In [20]:
df_news[df_news.duplicated()]

,event_id,text
513,9,"Коммерсантъ. Новости информ. центра, Москва, ..."
518,9,"Коммерсантъ. Новости информ. центра, Москва, ..."
522,9,"Коммерсантъ. Новости информ. центра, Москва, ..."
526,9,"Коммерсантъ. Новости информ. центра, Москва, ..."
528,9,"Коммерсантъ. Новости информ. центра, Москва, ..."
536,9,"Коммерсантъ. Новости информ. центра, Москва, ..."


### Часть 2 [3 балла] Кластеризация текстов
Любым известным вам алгоритмом найдите в коллекции новостей 28 кластеров. Используйте $tf-idf$ преобразование, сингулярное разложение, нормировку признакого пространства и любые другие техники обработки данных, которые вы считаете нужным. Используйте любые известные вам меры качества для оценки качества кластеризации. Покажите, как  $tf-idf$ преобразование или сингулярное разложение или любая другая использованная вами техника влияет на качество кластеризации. 

In [174]:
import warnings
warnings.filterwarnings('ignore') 

Используем Truncated SVD (a.k.a Latent Semantic Analysis):

In [16]:
words_list = [' '.join([word for word in sentence]) for sentence in texts]
ngram_range=(1,1)
n_clusters=28

In [14]:
for vectorizer, vectorizer_name in [
                        (TfidfVectorizer(ngram_range=ngram_range), 'TF-IDF Vectorizer'),
                          #(CountVectorizer(ngram_range=ngram_range), 'Count Vectorizer'), 
                          #(HashingVectorizer(ngram_range=ngram_range), 'Hashing Vectorizer')
                        ]:
    count_model = vectorizer
    X = count_model.fit_transform(words_list)
    svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
    U = svd.fit_transform(X)  
    for clusterizer, clusterizer_name in [
                                        #(SpectralClustering(n_clusters=n_clusters), 'Spectral Clustering'),
                                         #(KMeans(n_clusters=n_clusters), 'KMeans'),
                                         (AgglomerativeClustering(n_clusters=n_clusters), 'AgglomerativeClustering')
                                        ]:
        clusterizer_model = clusterizer.fit(U)
        print('Vectroizer: {}, Clusterizer: {} \nARI = {:0.2f}, V = {:0.2f}, completenss = {:0.2f}, homogenity = {:0.2f} \n======'.format(
                    vectorizer_name,
                    clusterizer_name,
                    adjusted_rand_score(clusterizer_model.labels_, df_news.event_id),
                    completeness_score(clusterizer_model.labels_, df_news.event_id),
                    v_measure_score(clusterizer_model.labels_, df_news.event_id),
                    homogeneity_score(clusterizer_model.labels_, df_news.event_id)))

Vectroizer: TF-IDF Vectorizer, Clusterizer: AgglomerativeClustering 
ARI = 0.46, V = 0.75, completenss = 0.74, homogenity = 0.74 


Используем Normalizer:

In [178]:
for vectorizer, vectorizer_name in [(TfidfVectorizer(ngram_range=ngram_range), 'TF-IDF Vectorizer'),
                          (CountVectorizer(ngram_range=ngram_range), 'Count Vectorizer'), 
                          (HashingVectorizer(ngram_range=ngram_range), 'Hashing Vectorizer')]:
    count_model = vectorizer
    X = count_model.fit_transform(words_list)
    normalizer = Normalizer()
    U = normalizer.fit_transform(X)
    for clusterizer, clusterizer_name in [(SpectralClustering(n_clusters=n_clusters), 'Spectral Clustering'),
                                         (KMeans(n_clusters=n_clusters), 'KMeans'),
                                         (AgglomerativeClustering(n_clusters=n_clusters), 'AgglomerativeClustering')]:
        clusterizer_model = clusterizer.fit(U.toarray())
        print('Vectroizer: {}, Clusterizer: {} \nARI = {:0.2f}, V = {:0.2f}, completenss = {:0.2f}, homogenity = {:0.2f} \n======'.format(
                    vectorizer_name,
                    clusterizer_name,
                    adjusted_rand_score(spectral.labels_, df_news.event_id),
                    completeness_score(spectral.labels_, df_news.event_id),
                    v_measure_score(spectral.labels_, df_news.event_id),
                    homogeneity_score(spectral.labels_, df_news.event_id)))

Vectroizer: TF-IDF Vectorizer, Clusterizer: Spectral Clustering 
ARI = 0.80, V = 0.91, completenss = 0.90, homogenity = 0.89 
Vectroizer: TF-IDF Vectorizer, Clusterizer: KMeans 
ARI = 0.80, V = 0.91, completenss = 0.90, homogenity = 0.89 
Vectroizer: TF-IDF Vectorizer, Clusterizer: AgglomerativeClustering 
ARI = 0.80, V = 0.91, completenss = 0.90, homogenity = 0.89 
Vectroizer: Count Vectorizer, Clusterizer: Spectral Clustering 
ARI = 0.80, V = 0.91, completenss = 0.90, homogenity = 0.89 
Vectroizer: Count Vectorizer, Clusterizer: KMeans 
ARI = 0.80, V = 0.91, completenss = 0.90, homogenity = 0.89 
Vectroizer: Count Vectorizer, Clusterizer: AgglomerativeClustering 
ARI = 0.80, V = 0.91, completenss = 0.90, homogenity = 0.89 
Vectroizer: Hashing Vectorizer, Clusterizer: Spectral Clustering 
ARI = 0.80, V = 0.91, completenss = 0.90, homogenity = 0.89 


KeyboardInterrupt: 

### Часть 3 [5 баллов] Кластеризация текстов (продолжение)
Задайте число кластеров заведомо меньше 28 (например, 5 кластеров). Повторно кластеризуйте новости. Можете ли вы проинтепретировать полученные кластеры? Получается ли с помощью кластерного анализа определить в каких сферах жизни (например, культура, политика, спорт) произошли события?

In [68]:
vectorizer = TfidfVectorizer(ngram_range=ngram_range)
X = vectorizer.fit_transform(words_list)
svd = TruncatedSVD(n_components=5, n_iter=7, random_state=42)
U = svd.fit_transform(X) 
clusterizer = SpectralClustering(n_clusters=5)
clusterizer_model = clusterizer.fit(U)

In [69]:
labels = clusterizer_model.labels_

In [70]:
events_merged = []

for event in range(5):
    events = set([df_news.event_id.values[i] for i, x in enumerate(labels) if x == event])
    events_merged.append(events)

In [71]:
for events_set in events_merged:
    for event in events_set:
        print(df_events.loc[df_events.index == event]['name'].values[0])
    print('=======')

Дональд Трамп вступил в должность президента США.
CNN показала фильм «Владимир Путин — самый влиятельный человек в мире».
Юлию Самойлову не пустили на евровидении в Киеве
В центре Киева был убит бывший депутат Госдумы РФ от КПРФ Денис Вороненков
Тиллерсон посещает Москву и встречается с Путиным
Несанкционированные акции в Москве апрель
Путин и Меркель в Сочи
Победа Макрона во Франции
Горячая линия Президента Путина
Саммит G20
Единый день голосования
Дональд Трамп вступил в должность президента США.
Скоропостижно скончался постпред России при ООН Виталий Чуркин.
Вышел фильм Навального «он Вам не димон»
Умер Дэвид рокфеллер
теракт произошел в центре Лондона
Юлию Самойлову не пустили на евровидении в Киеве
В центре Киева был убит бывший депутат Госдумы РФ от КПРФ Денис Вороненков
Митинг в москве против коррупции
SpaceX впервые в истории запустила и посадила уже летавшую ракету-носитель
Умер Евгений Евтушенко
Премьер Медведев выступает перед депутатами Госдумы с отчетом об итогах работы пр

In [74]:
texts = [[word for word in document.lower().split() if word not in stop] for document in df_news.text[:100]]
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
lda = LdaModel(corpus, id2word=dictionary, num_topics=10)

In [75]:
lda.print_topics(5)

[(2,
  '0.019*"собора" + 0.013*"рпц" + 0.011*"-" + 0.010*"исаакиевского" + 0.009*"передачи" + 0.008*"против" + 0.007*"собор" + 0.006*"января" + 0.006*"13" + 0.005*"передаче"'),
 (3,
  '0.012*"собора" + 0.011*"-" + 0.008*"передачи" + 0.007*"рпц" + 0.005*"исаакиевского" + 0.005*"это" + 0.004*"церкви" + 0.004*"передаче" + 0.004*"января" + 0.004*"собор"'),
 (6,
  '0.022*"собора" + 0.018*"исаакиевского" + 0.016*"передачи" + 0.011*"рпц" + 0.011*"против" + 0.011*"января" + 0.010*"13" + 0.010*"-" + 0.008*"2017" + 0.008*"собор"'),
 (0,
  '0.009*"-" + 0.009*"собора" + 0.008*"передачи" + 0.007*"против" + 0.006*"рпц" + 0.006*"собор" + 0.006*"исаакиевского" + 0.005*"января" + 0.005*"13" + 0.005*"это"'),
 (7,
  '0.014*"исаакиевского" + 0.013*"рпц" + 0.012*"собора" + 0.011*"-" + 0.010*"передачи" + 0.009*"против" + 0.008*"января" + 0.006*"13" + 0.006*"акция" + 0.005*"исаакиевский"')]